In [1]:
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [5]:
data = genfromtxt("./running_clean_nov.csv", delimiter=',')
lon=data[:,0]
lat=data[:,1]
hour=data[:,3]
minu=data[:,4]
seg=data[:,5]

In [ ]:

{}


sonpy? para los mapas